In [28]:
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np
import re
import progressbar
from itertools import chain
from multiprocessing import Pool
import functools
from tqdm import tqdm, trange

In [13]:
html = "http://www.dfshop.com/dfshop/wsTeamShop.wsc/pTsHome.p?session=dmlbhleghOiasdvbkcabPMCQbIdbfach&f=Home"

In [14]:
html

'http://www.dfshop.com/dfshop/wsTeamShop.wsc/pTsHome.p?session=dmlbhleghOiasdvbkcabPMCQbIdbfach&f=Home'

In [15]:
def remove_duplicates(l):
    return list(set(l))

In [24]:
def flatten(list_):
    if any(isinstance(i, list) for i in list_):
        list_ = chain.from_iterable(list_)
    return list_

In [25]:
def get_links(htmls, pattern):
    if isinstance(htmls, str):
        htmls = [htmls]
    container = []
    for i in htmls:
        source = urllib.request.urlopen(i)
        soup = bs.BeautifulSoup(source, 'lxml')

        for link in soup.findAll('a'):
            l = link.get('href')
            if re.match(pattern, l):
                container.append("http://www.dfshop.com"+l)

    return remove_duplicates(container)

In [32]:
def get_article(html):
    pool = Pool(6)
    iter_cat = functools.partial(get_links, pattern=".+cat=")
    iter_art = functools.partial(get_links, pattern=".+art=")
    
    parents            = flatten(get_links(html, pattern=".+cat="))
    children           = flatten(pool.map(iter_cat, parents))
    grandchildren      = flatten(pool.map(iter_cat, children))
    grandgrandchildren = flatten(pool.map(iter_cat, grandchildren))
    arts               = flatten(pool.map(iter_art, grandgrandchildren))
    return list(arts)

In [33]:
arts = get_article(html)


0it [00:00, ?it/s]

TypeError: object of type 'itertools.chain' has no len()